In [2]:
import pandas as pd
import numpy as np
import re
from pandas.api.types import is_string_dtype, is_numeric_dtype
import warnings
from pdb import set_trace

from torch import nn, optim, as_tensor
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.nn.init import *

import sklearn
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv("data/prepared_train_data.csv")
test = pd.read_csv("data/prepared_test_data.csv")
y = np.loadtxt('data/target_train_data.csv')

# Impute null values

In [4]:
train_imputed = train.fillna(train.mean())
test_imputed = test.fillna(train.mean())

In [5]:
train_imputed['f0'] = train_imputed['f0'] - 1000

In [6]:
train_imputed['f12_dec'] = np.log(train_imputed['f12_dec'])

In [7]:
test_imputed['f0'] = test_imputed['f0'] - 1000

In [8]:
test_imputed['f12_dec'] = np.log(test_imputed['f12_dec'])

In [9]:
train_imputed

,Unnamed: 0,target,f0,f1,f2,f3,f5,f6,f7,f9,...,f21_dnd,f21_n,f21_nd,f21_nnd,f21_nnx,f21_nx,f21_x,f21_xbx,f21_xnx,f12_dec
0,0,0,13.1,0,5.0190,26.1,11.100000,33.0,11.0,0.0,...,0,0,1,0,0,0,0,0,0,24.656501
1,1,0,10.2,1,5.0142,10.4,7.637312,46.0,17.0,0.0,...,0,0,0,0,0,1,0,0,0,23.480187
2,2,0,14.4,2,5.0280,12.3,7.637312,31.0,13.0,1.0,...,0,0,0,0,0,1,0,0,0,24.378407
3,3,0,19.2,3,5.0335,23.0,11.600000,41.0,9.0,0.0,...,1,0,0,0,0,0,0,0,0,24.803148
4,4,0,27.6,4,5.0129,7.3,7.300000,28.0,15.0,1.0,...,0,0,0,0,0,0,0,0,0,24.891308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49486,49995,1,30.1,40,5.0161,14.4,7.637312,30.0,15.0,0.0,...,0,1,0,0,0,0,0,0,0,24.616978
49487,49996,1,17.9,22,5.0319,23.4,1.800000,39.0,9.0,0.0,...,0,0,0,0,0,0,0,0,1,24.155893
49488,49997,0,21.2,13,5.0247,14.0,12.200000,39.0,19.0,0.0,...,0,0,1,0,0,0,0,0,0,23.921144
49489,49998,0,12.4,0,5.0219,29.9,9.500000,52.0,20.0,0.0,...,0,0,0,1,0,0,0,0,0,22.586714


In [10]:
test_imputed

,Unnamed: 0,f0,f1,f2,f3,f5,f6,f7,f9,f10,...,f21_dnd,f21_n,f21_nd,f21_nnd,f21_nnx,f21_nx,f21_x,f21_xbx,f21_xnx,f12_dec
0,0,24.4,15,5.0256,6.6,0.800000,20.0,2.0,0.0,0.200000,...,0,0,0,0,0,0,0,0,0,24.939016
1,1,29.0,3,5.0289,5.6,9.000000,28.0,0.0,0.0,1.800000,...,0,0,0,0,0,0,0,0,0,23.923797
2,2,16.7,10,5.0081,22.3,7.637312,31.0,6.0,0.0,5.467156,...,0,0,0,0,0,0,0,0,0,24.093026
3,3,9.3,33,5.0168,11.3,7.637312,31.0,11.0,1.0,5.467156,...,1,0,0,0,0,0,0,0,0,24.107977
4,4,27.1,37,5.0205,12.6,12.600000,22.0,6.0,0.0,5.000000,...,0,0,0,0,0,0,0,0,0,24.278093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,3.6,5,5.0143,23.6,0.000000,76.0,39.0,1.0,5.467156,...,0,0,0,0,0,1,0,0,0,24.563382
49996,49996,0.3,14,5.0098,13.9,9.200000,87.0,30.0,0.0,2.600000,...,0,0,0,0,0,0,0,1,0,24.629913
49997,49997,22.5,4,5.0189,10.7,10.300000,28.0,7.0,0.0,5.600000,...,0,0,0,0,0,0,0,0,0,24.807665
49998,49998,14.6,24,5.0263,15.0,7.637312,31.0,13.0,1.0,2.800000,...,0,0,0,0,0,0,0,0,0,24.301557


In [11]:
train_imputed.to_csv("data/imputed_train_data.csv")
test_imputed.to_csv("data/imputed_test_data.csv")

In [12]:
train_tensor = torch.tensor(train_imputed.drop('target', axis=1).values)
test_tensor = torch.tensor(test_imputed.values)
target_tensor = torch.tensor(y.astype(int))

In [13]:
target_tensor.shape

torch.Size([49491])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(train_tensor, target_tensor, test_size=0.33, random_state=7)

In [16]:
X_train.shape

torch.Size([33158, 85])

In [17]:
y_train.shape

torch.Size([33158])

In [18]:
class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()
      self.conv1 = nn.Linear(2818430, 256)
      self.conv2 = nn.Linear(256, 128)
      self.dropout1 = nn.Dropout(0.25)
      self.dropout2 = nn.Dropout(0.5)
      self.fc1 = nn.Linear(128, 128)
      self.fc2 = nn.Linear(128, 33158)

    # x represents our data
    def forward(self, x):
      # Pass data through conv1
      x = self.conv1(x)
      # Use the rectified-linear activation function over x
      x = F.relu(x)

      x = self.conv2(x)
      x = F.relu(x)

      # Pass data through dropout1
      x = self.dropout1(x)
      # Pass data through fc1
      x = self.fc1(x)
      x = F.relu(x)
      x = self.dropout2(x)
      x = self.fc2(x)

      # Apply softmax to x
      output = F.softmax(x, dim=0)
      return output

In [19]:
net = Net()
net = net.double()

In [20]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_tensor, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = X_train.flatten().double()
        labels = y_train.double()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.double())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
labels.shape